In [9]:
import os
import inspect
import json
import io

In [2]:
def log(s):
    print("::>",inspect.stack()[1][3]+':',s)

In [59]:
def build_docker(path_to_dockerfile,image_name,image_tag):
    command='docker build -t %s:%s %s'%(image_name,image_tag,path_to_dockerfile)
    log(command)
    os.system(command)

In [60]:
def usertag_docker_image(image_name, image_tag, username):
    command='docker tag %s:%s %s/%s:%s'%(image_name,image_tag, username, image_name,image_tag)
    log(command)
    os.system(command)

In [61]:
def push_docker_image(image_name, image_tag, username, password):
    command='docker login -u "%s" -p "%s" docker.io'%(username, password)
    log(command)
    os.system(command)
    command='docker push %s/%s:%s'%(username, image_name,image_tag)
    log(command)
    os.system(command)

In [54]:
def replace_write_file(placeholder, replace_content, path, file_name,new_path):
    file = open(path+"/"+file_name,mode='r')
    file_content = file.read()
    file.close()

    file_content=file_content.replace(placeholder, replace_content)

    new_path=new_path+"/"+file_name
    file = open(new_path, "w")
    file.write(file_content)
    file.close()

In [52]:
def change_docker_k8_configs(k8_configs_dict,docker_username, image_name, image_tag):
    for config_type in k8_configs_dict: # dict is like filename.yaml: path, where to put final file
        docker_image = 'image: %s/%s:%s'%(docker_username,image_name,image_tag)
        log("Adding "+docker_image+" -> "+k8_configs_dict[config_type]+"/"+config_type)
        replace_write_file("{{DOCKER}}", docker_image, './k8_configs_templates',config_type,k8_configs_dict[config_type])

In [24]:
!cat ./common_server_details.json

{
    "docker_image_path": "/path/to/dockerfile/folder",
    "docker_image_name": "server_image",
    "docker_image_tag": "0.0.1"
}

In [3]:
def read_details():
    common_server_details = json.load(open('./common_server_details.json'))
    decoder_server_details = json.load(open('./decoder_server_details.json'))
    docker_hub_details = json.load(open('./docker_hub_details.json'))
    common_k8_details = json.load(open('./common_k8_details.json'))
    decoder_k8_details = json.load(open('./decoder_k8_details.json'))

    return common_server_details,decoder_server_details,docker_hub_details,common_k8_details,decoder_k8_details

In [41]:
def apply_k8_configs(k8_details):
    for k8 in k8_details:
        config_path=k8_details[k8]+"/"+k8
        command='kubectl apply -f %s'%(config_path)
        log(command)
        os.system(command)


In [3]:
!kubectl get pods -o wide

NAME                                   READY   STATUS    RESTARTS   AGE   IP           NODE       NOMINATED NODE   READINESS GATES
counting-deployment-7bdbfc49f6-26q7g   1/1     Running   0          31m   10.244.2.3   worker02   <none>           <none>
decoder-deployment-5856556d9b-2jzd9    1/1     Running   0          31m   10.244.3.4   worker03   <none>           <none>
detector-deployment-8545575dbf-vsn7s   1/1     Running   0          31m   10.244.3.3   worker03   <none>           <none>
resize-deployment-57d965d6bf-w87wg     1/1     Running   0          31m   10.244.5.3   worker05   <none>           <none>


In [27]:
def get_decoder_ip():
    """
    TODO: Write this
    """
    ip=None
    raw_data=os.popen("kubectl get pods -o wide").read()
    buf = io.StringIO(raw_data)
    for line in buf.readlines():
        if line.split()[0].split("-")[0]=='decoder':
            ip=line.split()[5]
            log("Decoder ip is: "+ip)
            break
    if ip!=None:
        return ip
    else:
        raise Exception("Cannot find decoder IP, is decoder running?")


In [28]:
get_decoder_ip()

10.244.3.4


'0.0.0.0'

In [ ]:
def make_decoder_listen():
    decoder_ip=get_decoder_ip()
    command="curl "+decoder_ip+":5057/add-decode-task/?source=rtmp%3A%2F%2F130.245.127.103%3A1935%2Flive%2Fsample_2"
    log("Curling the decoder:" + command)

In [6]:
build_docker('/root/home', 'server_something', "0.0.2")
usertag_docker_image('server','0.0.1','manavjeet')
push_docker_image('server','0.0.1','manavjeet','password')

::> build_docker: docker build -t server_something:0.0.2 /root/home
::> usertag_docker_image: docker tag server:0.0.1 manavjeet/server:0.0.1
::> push_docker_image: docker login -u "manavjeet" -p "password" docker.io
::> push_docker_image: docker push manavjeet/server:0.0.1


In [6]:
common_server_details,decoder_server_details,docker_hub_details,common_k8_details,decoder_k8_details=read_details()

In [55]:

change_docker_k8_configs(k8_details,'manavJEET',"SERVER","0.1.1")

::> change_docker_k8_configs: Adding image: manavJEET/SERVER:0.1.1 -> ../k8s_configurations/counting_deployment.yaml


In [42]:
apply_k8_configs(k8_details)

::> apply_k8_configs: kubectl apply -f ../k8_configurations/counting_deployment.yaml


In [1]:
# GLOBAL VARS
class vars:
    def __init__(self):
        self.BUILD_COMMON_IMAGE=False
        self.BUILD_DECODER_IMAGE=True
        

GLOBAL=vars()

In [2]:
def main():
    common_server_details,decoder_server_details,docker_hub_details,k8_details=read_details()

    if GLOBAL.BUILD_COMMON_IMAGE:
        #Building and pushing the common docker image
        build_docker(common_server_details["docker_image_path"], common_server_details["docker_image_name"]\
            , common_server_details["docker_image_tag"])
        usertag_docker_image(common_server_details["docker_image_name"],common_server_details["docker_image_tag"]\
            ,docker_hub_details['uname'])
        push_docker_image(common_server_details["docker_image_name"],common_server_details["docker_image_tag"]\
            ,docker_hub_details['uname'],docker_hub_details['passwd'])

    if GLOBAL.BUILD_DECODER_IMAGE:
        #Building and pushing the common docker image
        build_docker(decoder_server_details["docker_image_path"], decoder_server_details["docker_image_name"]\
            , decoder_server_details["docker_image_tag"])
        usertag_docker_image(decoder_server_details["docker_image_name"],decoder_server_details["docker_image_tag"]\
            ,docker_hub_details['uname'])
        push_docker_image(decoder_server_details["docker_image_name"],decoder_server_details["docker_image_tag"]\
            ,docker_hub_details['uname'],docker_hub_details['passwd'])

    # making k8 configs using common server
    change_docker_k8_configs(common_k8_details,docker_hub_details['uname'],common_server_details["docker_image_name"]\
        ,common_server_details["docker_image_tag"])

    # making k8 config using decoder server
    change_docker_k8_configs(decoder_k8_details,docker_hub_details['uname'],decoder_server_details["docker_image_name"]\
        ,decoder_server_details["docker_image_tag"])


In [62]:
main()

::> build_docker: docker build -t server_o:0.0.2 ..
